In [1]:
from CogVideoX_ti2v import *

embed_dim = 1920
text_embed_dim = 4096
max_text_seq_length = 226
f, c, w, h = 49, 16, 90, 60
latent_f = (f - 1) // 4 + 1
time_embed_dim = 512

video = torch.randn(2, latent_f, c, h, w)
text_embedding = torch.randn(2, max_text_seq_length, text_embed_dim)
img_embedding = torch.randn(2, max_text_seq_length, text_embed_dim)
timestep = torch.randn(2, )
attention_head_dim = 64
image_rotary_emb = [torch.randn(latent_f * w * h // 4, attention_head_dim), torch.randn(latent_f * w * h // 4, attention_head_dim)]
# cog_patch_i2v = CogVideoXPatchEmbed_TI2V(2, in_channels= 16, embed_dim=embed_dim, text_embed_dim=text_embed_dim,
#         sample_width=w, sample_height=h, sample_frames=f, temporal_compression_ratio=4, max_text_seq_length=max_text_seq_length, use_positional_embeddings=True,
#         image_embedding_dim=text_embed_dim, max_image_seq_length=max_text_seq_length)
# embeds = cog_patch_i2v(text_embedding, video, image_embeds=img_embedding)
# print(embeds.shape)

cogvideo_ti2v = CogVideoXTransformer3D_TI2V(
    num_attention_heads=30, attention_head_dim=64,
    in_channels=c, sample_width=w, sample_height=h, sample_frames=f, patch_size=2, temporal_compression_ratio=4,
    text_embed_dim=text_embed_dim, max_text_seq_length=max_text_seq_length,
    image_embedding_dim=text_embed_dim, max_image_seq_length=max_text_seq_length,
    time_embed_dim=time_embed_dim,
    num_layers=2, use_rotary_positional_embeddings=True, use_learned_positional_embeddings=True, use_decoupled_module=True)



In [ ]:
pred = cogvideo_ti2v(hidden_states=video, encoder_hidden_states=text_embedding, timestep=timestep,
    encoder_hidden_states_image=img_embedding, image_rotary_emb=image_rotary_emb)
print(pred.sample.shape)

In [5]:
import re
train_layer_name_lst = []
# print(cogvideo_ti2v.state_dict().keys())
attn_train_lst = ['to_q', 'to_k', 'to_v']       #注意力层可为调: ['to_q', 'to_k', 'to_v', 'norm_q','norm_k','norm_v','to_out']
# for layer_name in cogvideo_ti2v.state_dict().keys():
#     print(layer_name)
for layer_name, states_params in cogvideo_ti2v.state_dict().items():
    if layer_name.startswith('patch_embed.image_proj'):
        train_layer_name_lst.append(layer_name)
    if layer_name.startswith('decouple_module'):
        train_layer_name_lst.append(layer_name)
    if layer_name.startswith('transformer_blocks'):
        if re.search(r'transformer_blocks\.\d+\.norm1\.linear2', layer_name):
            train_layer_name_lst.append(layer_name)
        elif re.search(r'transformer_blocks\.\d+\.norm2\.linear2', layer_name):
            train_layer_name_lst.append(layer_name)
        elif re.search(r'transformer_blocks\.\d+\.attn1\.', layer_name):  # 注意力层
            for attn in attn_train_lst:
                if attn in layer_name:
                    train_layer_name_lst.append(layer_name)
                    break
print(train_layer_name_lst)

['patch_embed.image_proj.weight', 'patch_embed.image_proj.bias', 'transformer_blocks.0.norm1.linear2.weight', 'transformer_blocks.0.norm1.linear2.bias', 'transformer_blocks.0.attn1.to_q.weight', 'transformer_blocks.0.attn1.to_q.bias', 'transformer_blocks.0.attn1.to_k.weight', 'transformer_blocks.0.attn1.to_k.bias', 'transformer_blocks.0.attn1.to_v.weight', 'transformer_blocks.0.attn1.to_v.bias', 'transformer_blocks.0.norm2.linear2.weight', 'transformer_blocks.0.norm2.linear2.bias', 'transformer_blocks.1.norm1.linear2.weight', 'transformer_blocks.1.norm1.linear2.bias', 'transformer_blocks.1.attn1.to_q.weight', 'transformer_blocks.1.attn1.to_q.bias', 'transformer_blocks.1.attn1.to_k.weight', 'transformer_blocks.1.attn1.to_k.bias', 'transformer_blocks.1.attn1.to_v.weight', 'transformer_blocks.1.attn1.to_v.bias', 'transformer_blocks.1.norm2.linear2.weight', 'transformer_blocks.1.norm2.linear2.bias', 'decouple_module.norm1.weight', 'decouple_module.norm1.bias', 'decouple_module.norm2.weight

In [1]:
# vit
import torch
import torch.nn as nn  # 网络
import torch.nn.functional as F  # 激活函数
import torchvision.datasets  # 样本
from torch.utils import *  # 图案管理
from torch.utils import data  # 预处理数据集
from torchvision import transforms  # 工具
import torch.optim as optim
from tqdm import tqdm  # 进度条
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from transformers import CLIPVisionModelWithProjection


embeddings = torch.randn(1, 10, 1280)
image_encoder = CLIPVisionModelWithProjection.from_pretrained(r'E:\PythonLearn\work\SSH_Connect\Autodl\under2postgraudate\Video-Generation-field\Ours\Multiple scene\excluded_dir\local_model\model_dir\CLIP-ViT-H-14-laion2B-s32B-b79K', local_files_only=True, use_safetensors=True)
embeddings = image_encoder.vision_model.pre_layrnorm(embeddings)
encoder_outputs = image_encoder.vision_model.encoder(
        inputs_embeds=embeddings,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None)
# last_hidden_state:[B,59,1280], [f_s,f_p]=transfomer_blocks([f_s,f_p])
last_hidden_state = encoder_outputs[0]
# 取出style embedding f_s [B,3,1280]
pooled_output = last_hidden_state[:, :3, :]
# 层归一化  pooled_output:[B,3,1280]
pooled_output = image_encoder.vision_model.post_layernorm(pooled_output)
out = image_encoder.visual_projection(pooled_output)


In [2]:
print(out.shape)

torch.Size([1, 3, 1024])
